In [1]:
import openai
import json
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
import concurrent.futures
import random


In [2]:
# https://academic.oup.com/poq/article/86/S1/523/6623490
# https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/poq/86/S1/10.1093_poq_nfac022/3/nfac022_supplementary_data.pdf?Expires=1700072808&Signature=iZxE7ZOPWwKfHDQe3Ub8I4JCuTKurFmEcwACsTgNr0eN4MajtWFCsH~mRNIWk27FOi7DFqegt9p9Ecyu8lblLDzokLZN37AHwCo61V4GfsAkOJfRTLVYAVZipfoYRk78vSLN10RXex~PvvTP8j5Cq8UnWALC7t2FfZQw35MRFpEzTwqLya0f-11Lw2bfEhfG634P7rV7CVzREvjVydegym~QfA0HILq9M8mf1lk-TF4lA7XWr8rCxxSW~PW5Ts732EzBWLX4YijGj8W098O-SVi63K-teF2hCqqKpPh3mwWy5XMmGhLhBRUsZRXO5Y~5jVTBGybYPbxrH1YmkANN2A__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA

In [3]:
# Lists of attributes
white_signaling_names = [
    "Daniel Nash",
    "Mathew Roberts",
    "Alex Steele",
    "Nicholas Austin",
    "Zachary Fitzpatrick",
    "Christopher Schmidt",
    "Ryan Thompson",
    "Timothy Bartlett",
    "Corey Kennedy",
    "Garrett Riddle",
    "Austin Walsh"
]

black_signaling_names = [
    "DeShawn Jackson",
    "Tyrone Brown",
    "Darius Thomas",
    "Jamal Jones",
    "Terrell Turner",
    "Trevon Williams",
    "DeAndre Wilkins",
    "Darnell Haynes",
    "Jalen Washington",
    "Marquis Harris",
    "Malik Johnson"
]

names = white_signaling_names + black_signaling_names

# Defendant's Crime
convicted_of_crime = [
    "Forging checks worth a total of $5,000",
    "Forging checks from 12 different individuals worth a total of $60,000",
    "Forging checks from 12 different individuals worth a total of $600,000",
    "Threatening to pull a gun during a dispute with another individual",
    "Firing a gun at another individual (but missing)",
    "Shooting another individual, causing serious injury to them",
    "Possessing 1/3 of an ounce of heroin",
    "Possessing 6.8 pounds of heroin"
]

# Past Criminal History (Incarcerations)
past_criminal_history = [
    "No prior convictions",
    "Previous conviction, sentenced to 6 months",
    "Previous conviction, sentenced to 2 years"
]

# On Probation When They Committed the Crime
on_probation = ["Yes", "No"]

employed_at_least_part_time = ["Yes", "No"]

age_categories = ["15-17", "18-21", "22-30"]


import random

def generate_random_person():
    # Randomly select one item from each list
    name = random.choice(white_signaling_names + black_signaling_names)
    crime = random.choice(convicted_of_crime)
    history = random.choice(past_criminal_history)
    probation = random.choice(on_probation)
    employment = random.choice(employed_at_least_part_time)
    age = random.choice(age_categories)

    # Create a dictionary to represent the random person
    random_person = {
        "Name": name,
        "Crime": crime,
        "Past Criminal History": history,
        "Currently on Probation": probation,
        "Currently Employed at Least Part Time": employment,
        "Age": age
    }

    return random_person



# Example usage
random_person = generate_random_person()
attributes = list(random_person.keys())

In [5]:
generate_random_person()


{'Name': 'Nicholas Austin',
 'Crime': 'Forging checks worth a total of $5,000',
 'Past Criminal History': 'Previous conviction, sentenced to 2 years',
 'Currently on Probation': 'Yes',
 'Currently Employed at Least Part Time': 'No',
 'Age': '22-30'}

In [6]:
study_response_form = {
    "name": "study_response_form",
    "description": '''What sentence do you think should be given? If sentence to jail or prison, how long?''',
    "parameters": {
        "type": "object",
        "properties": {
            "sentence_option": {
                "type": "string",
                "description": '''What sentence do you think should be given? Respond with only the letter.         A) Probation (no time in prison), 
        B) Less than 1 year. If you select this option, list the number of months (1 to 11 months). 
        C) 1 or more years.  If you select this option, list the number of years (1 or more years)
        D) Life (in prison).'''
            },
            
            "duration": {
                "type": "string",
                "description": '''If the sentence is not probation, how many months or years? Include the word months or years.'''
            },
            "Explanation": {
                "type": "string",
                "description": '''Explain your sentence recommendation.'''
            },

            
        },
        "required": ["sentence_option"]
    }
}

In [7]:
def load(string):
    return json.loads(string)

@retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(2))
def what_sentence(n=3):
    random.shuffle(attributes)
    i1 = generate_random_person()
    i1s = '\n'.join([f'''* {k}: {i1[k]}''' for k in attributes])
    
    
    system = '''This study considers criminal sentencing. For the next few minutes, we are going to ask you to act as if you were an criminal judge. We will provide you with several pieces of information about people who might be convicted of a crime. For each pair of people, please indicate what sentence should be given. This exercise is purely hypothetical.  Even if you aren’t entirely sure, please indicate your preference.'''
    
    q = f'''
        This study considers criminal sentencing. For the next few minutes, we are going to ask you to act as if you were an criminal judge. We will provide you with several pieces of information about people who might be convicted of a crime. For each pair of people, please indicate what sentence should be given. This exercise is purely hypothetical.  Even if you aren’t entirely sure, please indicate your preference.        
        Pleae read the descriptions of the hypothetical individual carefully . 
    
        {i1s}
        
        
        What sentence do you think should be given to {i1['Name']}?” 
        A) Probation (no time in prison), 
        B) Less than 1 year. If you select this option, list the number of months (1 to 11 months). 
        C) 1 or more years.  If you select this option, list the number of years (1 or more years)
        D) Life (in prison).
        
'''

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content":  q }
        ]
    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        # model='gpt-4',
        functions=[study_response_form],
        n=n,
        messages=messages)
    
    df = pd.DataFrame([load(r['message']['function_call']['arguments']) for r in response['choices']])
    
    for a in i1.keys():
        df[a] = i1[a]
    return df

    melted_df = pd.melt(df, id_vars='preferred_immigrant_ID',
                        var_name='immigrant_ID',
                        value_name='immigrant_score',
                        ignore_index=False)

    melted_df['immigrant_ID'] = melted_df['immigrant_ID'].str.split('_').str[1]
    
    melted_df['immigrant_ID'] = melted_df['immigrant_ID'].astype(int)
    melted_df = melted_df.merge(idf, left_on='immigrant_ID', right_index=True, how='outer')
    melted_df = melted_df.sort_values(by='immigrant_ID').sort_index()
    return melted_df

In [8]:
response = what_sentence()
response

RetryError: RetryError[<Future at 0x137b0a440 state=finished raised APIRemovedInV1>]

In [8]:
results = []

In [19]:
num_iterations = 80
from concurrent.futures import ThreadPoolExecutor

try:
    print(len(pd.concat(results)))
except (NameError, ValueError):
    results = []
    print(0)
    
# Function to run eval_tactic and append the result to the list
def run_and_append_result(_):
    result = what_sentence()
    results.append(result)

for i in range(0,20):
    print(i)
    with ThreadPoolExecutor() as executor:
        # Map the function over the range of iterations
        executor.map(run_and_append_result, range(num_iterations))
    

4788
0
1



KeyboardInterrupt



In [20]:
rdf = pd.concat(results)
print(len(rdf))
rdf = rdf[rdf['sentence_option'].isin(['A','B','C','D'])].copy()
rdf['Black Name'] = rdf['Name'].isin(black_signaling_names)
len(rdf)

5265


5057

In [21]:
rdf.to_json('conjoint_sentencing.json', orient='records')
rdf.to_csv('conjoint_sentencing.csv', index=False)

rdf.keys()

Index(['sentence_option', 'Name', 'Crime', 'Past Criminal History',
       'Currently on Probation', 'Currently Employed at Least Part Time',
       'Age', 'duration', 'Explanation', 'crime', 'age', 'prior_convictions',
       'currently_employed', 'Sentence_option', 'Duration', 'option',
       'senetence_option', 'se​​ntence_option', 'sent_Form', 'sentenc_option',
       'speech_option', 'senetnce_option', 'options', 'race',
       '1. sentence_option', '2. duration', '3. Explanation',
       'sentences_option', 'Unnamed', 'sentencing_option', 'hr_type',
       'sesxsentence_option', 'song_option', 'name', 'past_criminal_history',
       'currently_on_probation', 'Probanality', 'word', 'sentnence_option',
       'the_sentence_option', ' sentence_option', 'study_response_form',
       'option1', 'senence_option', 'require_response', 'refusal_dialog',
       'explanation_dialog', 'sentance_option', 'section', 'sentene_option',
       'question1', 'Sentence Option', '(sentence_option)',

In [12]:
pd.crosstab(rdf['Black Name'], rdf['sentence_option'])

sentence_option,A,B,C,D
Black Name,,,,
False,261,392,508,5
True,268,342,530,3


In [13]:
pd.crosstab(rdf['Black Name'], rdf['sentence_option'], normalize='index')

sentence_option,A,B,C,D
Black Name,,,,
False,0.223842,0.336192,0.435678,0.004288
True,0.234471,0.299213,0.463692,0.002625


In [14]:
pd.crosstab(rdf['Currently Employed at Least Part Time'], rdf['sentence_option'], normalize='index')

sentence_option,A,B,C,D
Currently Employed at Least Part Time,,,,
No,0.191928,0.324547,0.477759,0.005766
Yes,0.270320,0.310502,0.418265,0.000913


In [128]:
groups = rdf.groupby(level=0)

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over groups and apply pd.crosstab
for name, group in groups:
    cross_tab_result = pd.crosstab(group['Black Name'], group['sentence_option'], normalize='index')
    result_df = pd.concat([result_df, cross_tab_result])

print(result_df)

sentence_option         A         B         C         D
Black Name                                             
False            0.217489  0.331839  0.448430  0.002242
True             0.203271  0.350467  0.443925  0.002336
False            0.210762  0.340807  0.446188  0.002242
True             0.179907  0.322430  0.492991  0.004673
False            0.222222  0.331066  0.444444  0.002268
True             0.202326  0.320930  0.474419  0.002326


In [162]:
for attribute in [a for a in attributes if a !="Name"]:
    print(f'\n\n{attribute}')

    cross_tab_result = pd.crosstab(group[attribute], group['sentence_option'], normalize='index').sort_values(by='A')
    display(cross_tab_result)



Crime


sentence_option,A,B,C,D
Crime,,,,
"Forging checks from 12 different individuals worth a total of $600,000",0.000000,0.130178,0.869822,0.000000
"Forging checks from 12 different individuals worth a total of $60,000",0.029240,0.286550,0.684211,0.000000
"Shooting another individual, causing serious injury to them",0.055556,0.322222,0.616667,0.005556
Possessing 6.8 pounds of heroin,0.068966,0.235632,0.678161,0.017241
"Forging checks worth a total of $5,000",0.408759,0.532847,0.058394,0.000000
Firing a gun at another individual (but missing),0.428571,0.466165,0.105263,0.000000
Threatening to pull a gun during a dispute with another individual,0.492063,0.428571,0.079365,0.000000
Possessing 1/3 of an ounce of heroin,0.767196,0.201058,0.026455,0.005291




Past Criminal History


sentence_option,A,B,C,D
Past Criminal History,,,,
"Previous conviction, sentenced to 2 years",0.181193,0.298165,0.513761,0.006881
"Previous conviction, sentenced to 6 months",0.224344,0.369928,0.405728,0.000000
No prior convictions,0.410377,0.264151,0.320755,0.004717




Currently on Probation


KeyError: 'Currently on Probation'

In [129]:
groups = rdf.groupby('Crime')

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over groups and apply pd.crosstab
for name, group in groups:
    print(name)
    cross_tab_result = pd.crosstab(group['Black Name'], group['sentence_option'], normalize='index')
    display(cross_tab_result)

Firing a gun at another individual (but missing)


sentence_option,A,B,C
Black Name,,,
False,0.272189,0.562130,0.165680
True,0.238095,0.503401,0.258503


Forging checks from 12 different individuals worth a total of $60,000


sentence_option,A,B,C
Black Name,,,
False,0.016949,0.220339,0.762712
True,0.012658,0.215190,0.772152


Forging checks from 12 different individuals worth a total of $600,000


...

sentence_option,A,B,C,D
Black Name,,,,
False,0.019737,0.138158,0.828947,0.013158
True,0.054645,0.180328,0.748634,0.016393


Shooting another individual, causing serious injury to them


sentence_option,A,B,C,D
Black Name,,,,
False,0.012270,0.263804,0.723926,0.00000
True,0.050847,0.322034,0.621469,0.00565


Threatening to pull a gun during a dispute with another individual


sentence_option,A,B,C
Black Name,,,
False,0.388889,0.518519,0.092593
True,0.324841,0.554140,0.121019


In [131]:
for attribute in [a for a in attributes if a !="Name"]:
    print(f'\n\n{attribute}')
    groups = rdf.groupby(attribute)

    # Initialize an empty DataFrame to store the results
    result_df = pd.DataFrame()

    # Iterate over groups and apply pd.crosstab
    for name, group in groups:
        print(name)
        cross_tab_result = pd.crosstab(group['Black Name'], group['sentence_option'], normalize='index')
        display(cross_tab_result)



Crime
Firing a gun at another individual (but missing)


sentence_option,A,B,C
Black Name,,,
False,0.272189,0.562130,0.165680
True,0.238095,0.503401,0.258503


Forging checks from 12 different individuals worth a total of $60,000


sentence_option,A,B,C
Black Name,,,
False,0.016949,0.220339,0.762712
True,0.012658,0.215190,0.772152


Forging checks from 12 different individuals worth a total of $600,000


...

sentence_option,A,B,C
Black Name,,,
False,0.154839,0.423656,0.421505
True,0.140940,0.391499,0.467562




On Probation
No


sentence_option,A,B,C,D
Black Name,,,,
False,0.166419,0.365527,0.468053,0.000000
True,0.121439,0.343328,0.532234,0.002999


Yes


sentence_option,A,B,C,D
Black Name,,,,
False,0.268182,0.303030,0.424242,0.004545
True,0.274637,0.318255,0.403877,0.003231


In [138]:
pd.crosstab(rdf['On Probation'], rdf['sentence_option'])

sentence_option,A,B,C,D
On Probation,,,,
No,193,475,670,2
Yes,347,397,530,5


In [108]:
pd.crosstab(rdf['preferred'], rdf['Employment Plan'], normalize='columns').T.sort_values(by=False)

preferred,False,True
Employment Plan,,
Has a contract with a U.S. employer,0.375951,0.624049
Will look for work after arriving in the U.S.,0.465625,0.534375
"Does not have a contract with a U.S. employer, but has done job interviews",0.584615,0.415385
Has no plans to look for work at this time,0.586817,0.413183
